In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_

from sklearn.metrics import accuracy_score, matthews_corrcoef, precision_score, recall_score, f1_score
from tensorboardX import SummaryWriter
from multiprocessing import Pool

import os
import math
from os import listdir, makedirs

import numpy as np
import pickle
from tqdm import tqdm

In [ ]:
WORK_DIR = '../Data/bioc_FH_training/'
LOG_DIR  = '../log/bioc_FH_training/'

lmap = lambda fun, it: list(map(lambda x: fun(x), it))

In [ ]:
with open(WORK_DIR + 'dict.pkl','rb') as f:
    ent, pos, dep = pickle.load(f)

d_ent=len(ent)
d_pos=len(pos)
d_dep=len(dep)

In [ ]:
class Attention(nn.Module):
    
    def __init__(self,d_h):
        super(Attention, self).__init__()
        self.W_q=nn.Linear(in_features=d_h,out_features=d_h,bias=True)
        self.W_k=nn.Linear(in_features=d_h,out_features=d_h,bias=True)
        self.W_v=nn.Linear(in_features=d_h,out_features=d_h,bias=True)
        
    def forward(self,q,k,v,mask):
        Q=self.W_q(q)
        K=self.W_k(k)
        V=self.W_v(v)
        O=torch.matmul(Q,torch.transpose(K,-2,-1))/ math.sqrt(Q.size(-1))
        O=O.masked_fill(mask==0,-1e9)
        E=F.softmax(O,dim=-1)
        R=torch.matmul(E,v)
#        G=F.sigmoid(self.W_g(R))*R
        return R

In [ ]:
# -*- coding:utf-8 -*-

class RNN(nn.Module):
    def __init__(self, d_x, d_h, d_ent, d_pos, d_dep, cell=nn.GRU, rnn_layers=3, ffn_layers=5, dp=0.5):
        super(RNN, self).__init__()
        self.dropout = nn.Dropout(p=dp)
        #self.embedding = nn.Embedding(_weight=torch.tensor(emb,dtype=torch.float32),
        #                              num_embeddings=embedding.shape[0],
        #                              embedding_dim=embedding.shape[1])
        self.rnn_layers = rnn_layers
        self.d_h=d_h
        self.rnn = cell(input_size=d_x, hidden_size=d_h, num_layers=rnn_layers, batch_first=True, dropout=dp,bidirectional=True)
        self.ent_attn = Attention(d_h=d_h)
        self.pos_attn = Attention(d_h=d_h)
        self.dep_attn = Attention(d_h=d_h)
        
        self.hidden_ents = nn.ModuleList([nn.Linear(in_features=d_h, out_features=d_h) for _ in range(ffn_layers)])
        self.hidden_poss = nn.ModuleList([nn.Linear(in_features=d_h, out_features=d_h) for _ in range(ffn_layers)])
        self.hidden_deps = nn.ModuleList([nn.Linear(in_features=d_h, out_features=d_h) for _ in range(ffn_layers)])
        
        self.lm_ents = nn.ModuleList([nn.LayerNorm(d_h) for _ in range(ffn_layers)])
        self.lm_poss = nn.ModuleList([nn.LayerNorm(d_h) for _ in range(ffn_layers)])
        self.lm_deps = nn.ModuleList([nn.LayerNorm(d_h) for _ in range(ffn_layers)])
        
        self.f_ent_out = nn.Linear(in_features=d_h, out_features=d_ent)
        self.f_pos_out = nn.Linear(in_features=d_h, out_features=d_pos)
        self.f_dep_out = nn.Linear(in_features=d_h, out_features=d_dep)
        
        for p in self.parameters(): # initialize parameters
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    
    def forward(self, x, mask, hidden=None):
        rnn_out, _ = self.rnn(x, hidden)
        rnn_out = rnn_out[:, :, :self.d_h] + rnn_out[:, :, self.d_h:]
        
        ent_out_attn = self.ent_attn(q=rnn_out, k=rnn_out, v=rnn_out, mask=mask)
        ent_out = ent_out_attn
        for hl, lm in zip(self.hidden_ents, self.lm_ents):
            ent_out = self.dropout(lm(F.leaky_relu(hl(ent_out))))
        ent_out = self.f_ent_out(ent_out_attn + ent_out)
        
        pos_out_attn = self.pos_attn(q=rnn_out,k=rnn_out,v=rnn_out,mask=mask)
        pos_out = pos_out_attn
        for hl, lm in zip(self.hidden_poss, self.lm_poss):
            pos_out = self.dropout(lm(F.leaky_relu(hl(pos_out))))
        pos_out = self.f_pos_out(pos_out_attn + pos_out)
        
        dep_out_attn = self.dep_attn(q=rnn_out,k=rnn_out,v=rnn_out,mask=mask)
        dep_out = dep_out_attn
        for hl, lm in zip(self.hidden_deps, self.lm_deps):
            dep_out = self.dropout(lm(F.leaky_relu(hl(dep_out))))
        dep_out = self.f_dep_out(dep_out_attn + dep_out)
        
        return ent_out, pos_out, dep_out

In [ ]:
class Classifier(object):
    def __init__(self, model, dp=0.5,lr=1e-3):
        super(Classifier, self).__init__()
        self.model = model
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
    
    def train(self, X, mask, y_ent, y_pos, y_dep):
        x = torch.tensor(X, dtype=torch.float)
        attn_mask = (mask > 0).unsqueeze(1).repeat(1, mask.size(1), 1)
        loss_mask = mask
        y_true_ent = torch.tensor(y_ent, dtype=torch.long)
        y_true_pos = torch.tensor(y_pos, dtype=torch.long)
        y_true_dep = torch.tensor(y_dep, dtype=torch.long)
        self.model.train(True)
        self.optimizer.zero_grad()
        y_hat_ent,y_hat_pos,y_hat_dep = self.model(x,attn_mask)
        
        loss_ent = self.maskNLLLoss(y_hat_ent, y_true_ent, loss_mask)
        loss_pos = self.maskNLLLoss(y_hat_pos, y_true_pos, loss_mask)
        loss_dep = self.maskNLLLoss(y_hat_dep, y_true_dep, loss_mask)
        reg_lambda=1e-4
        l2_reg = 0
        
        for W in self.model.named_parameters(): # regularizer
            if "weight" in W[0]:
                l2_reg = l2_reg + W[1].norm(2)
        loss=loss_ent+loss_pos+loss_dep+l2_reg * reg_lambda
        loss.backward()
        clip_grad_norm_(self.model.parameters(), 1) # gradient clip -> avoid gradient explosion
        self.optimizer.step()
        y_hat_ent = y_hat_ent.topk(1)[1].squeeze(-1).detach().cpu()
        y_hat_pos = y_hat_pos.topk(1)[1].squeeze(-1).detach().cpu()
        y_hat_dep = y_hat_dep.topk(1)[1].squeeze(-1).detach().cpu()
        return loss_ent.item(), loss_pos.item(), loss_dep.item(), y_hat_ent.numpy(), y_hat_pos.numpy(), y_hat_dep.numpy()
    
    def test(self, X, mask, y_ent, y_pos, y_dep):
        self.model.eval()
        with torch.no_grad():
            x = torch.tensor(X, dtype=torch.float)
            attn_mask = (mask > 0).unsqueeze(1).repeat(1, mask.size(1), 1)
            loss_mask = mask
            y_true_ent = torch.tensor(y_ent, dtype=torch.long)
            y_true_pos = torch.tensor(y_pos, dtype=torch.long)
            y_true_dep = torch.tensor(y_dep, dtype=torch.long)
            y_hat_ent,y_hat_pos,y_hat_dep = self.model(x,attn_mask)
            
            loss_ent = self.maskNLLLoss(y_hat_ent, y_true_ent, loss_mask)
            loss_pos = self.maskNLLLoss(y_hat_pos, y_true_pos, loss_mask)
            loss_dep = self.maskNLLLoss(y_hat_dep, y_true_dep, loss_mask)
            
            y_hat_ent = y_hat_ent.topk(1)[1].squeeze(-1).detach().cpu()
            y_hat_pos = y_hat_pos.topk(1)[1].squeeze(-1).detach().cpu()
            y_hat_dep = y_hat_dep.topk(1)[1].squeeze(-1).detach().cpu()
            
            return loss_ent.cpu().item(), loss_pos.cpu().item(), loss_dep.cpu().item(), y_hat_ent.numpy(), y_hat_pos.numpy(), y_hat_dep.numpy()
    
    def inference(self, X, mask):
        self.model.eval()
        with torch.no_grad():
            x = torch.tensor(X, dtype=torch.float)
            attn_mask = (mask > 0).unsqueeze(1).repeat(1, mask.size(1), 1)
            y_hat_ent,y_hat_pos,y_hat_dep = self.model(x,attn_mask)
            y_hat_ent = y_hat_ent.topk(1)[1].squeeze(-1).detach().cpu()
            y_hat_pos = y_hat_pos.topk(1)[1].squeeze(-1).detach().cpu()
            y_hat_dep = y_hat_dep.topk(1)[1].squeeze(-1).detach().cpu()
            return y_hat_ent.numpy().flatten(), y_hat_pos.numpy(), y_hat_dep.numpy()
    
    def maskNLLLoss(self, logits, target, mask):
        logits=F.softmax(logits,dim=-1)
        crossEntropy = -torch.log(logits.gather(-1,target[:,:,None])).squeeze(-1)
        loss = crossEntropy.masked_select(mask).mean()
        return loss
    
    def load_model(self, model_path='./Model'):
        self.model = torch.load(model_path + '/model.pkl')
    
    def save_model(self, model_path='./Model'):
        if not os.path.exists(model_path):
            os.mkdir(model_path)
        torch.save(self.model, model_path + '/model.pkl')

def build_conv(n_gram=[1,3,5],in_channels=[300,300,300], out_channels=[50,50,50]):
    convs=nn.ModuleList([nn.Conv1d(in_channels=d_in,out_channels=d_out,kernel_size=n,padding=p,stride=1) for p,(d_in, d_out, n) in enumerate(zip(in_channels,out_channels,n_gram))])
    return convs
        
def pad(sents,dim=5):
    padded=[]
    max_leng=max(lmap(lambda x:x.shape[0],sents))
    for s in sents:
        if s.shape[0]<=max_leng:
            padded.append(np.concatenate((s,np.zeros((max_leng-s.shape[0],dim),dtype=np.int32))))
    return np.array(padded)

def unpad(y_true, y_hat):
    y_true = np.concatenate(y_true)
    y_hat = np.concatenate(y_hat)
    idx = (y_true != 0)
    y_true = y_true[idx]
    y_hat = y_hat[idx]
    return y_true, y_hat

def mask_unpad(mask, y_hat):
    y_hat = np.concatenate(y_hat)
    idx = (mask != 0)
    mask = mask[idx]
    y_hat = y_hat[idx]
    return y_hat

# Training

In [ ]:
model=RNN(d_x=1024,d_h=100,d_ent=d_ent,d_pos=d_pos,d_dep=d_dep,rnn_layers=1,ffn_layers=2)

In [ ]:
clf = Classifier(model=model,lr=1e-3)

In [ ]:
writer = SummaryWriter(log_dir=LOG_DIR)
train_size = 72

txt_files = [f for f in listdir(WORK_DIR) if f.endswith('.txt')]


emb_files = list(map(lambda x: x[:-3]+'pkl',txt_files))

labels=np.load(WORK_DIR + 'label.npy')

def read_emb_f(f):
    with open(WORK_DIR + f, 'rb') as emb_f:
        corpus = pickle.load(emb_f)
    return corpus

pool = Pool(processes=8) 
all_corpus = pool.map(read_emb_f,emb_files)

all_corpus = list(zip(all_corpus,labels))
train_corpus = all_corpus[:train_size]
test_corpus = all_corpus[train_size:]

#corpus = list(zip(emb_files,labels))

In [ ]:
batch_size=128

In [ ]:
for e in tqdm(range(20)): 
    np.random.shuffle(train_corpus)
    for fi, f in enumerate(train_corpus):
        
        corpus = f[0]
        label = lmap(lambda x: np.array(x), f[1])
        label = pad(label,dim=3)
        dl = []
        i = 0
        while i < label.shape[0]:
            if label.shape[0]==0: break
            x = corpus['elmo_representations'][0][i:i + batch_size]
            mask =  corpus['mask'].type(torch.uint8)[i:i + batch_size]
            y = label[i:i + batch_size]
            l1,l2,l3,y1,y2,y3=clf.train(x,mask,y_ent=y[:,:,0],y_pos=y[:,:,1],y_dep=y[:,:,2])
            i += batch_size
            dl.append(l1+l2+l3)
        writer.add_scalar('train loss',np.mean(dl),e*len(train_corpus)+fi)
        #print(f'train epoch: {e}, doc: {fi}, loss: {np.mean(dl):.2f}')
    print('-'*18)

    tall, pall = [], []
    for fi, f in enumerate(test_corpus):
        corpus = f[0]
        label = lmap(lambda x: np.array(x), f[1])
        label = pad(label,dim=3)
        pent = []
        ppos = []
        pdep = []
        tent = []
        tpos = []
        tdep = []
        dl = []
        i=0
        while i<label.shape[0]:
            if label.shape[0]==0:break
            x = corpus['elmo_representations'][0][i:i+batch_size]
            mask =  corpus['mask'].type(torch.uint8)[i:i+batch_size]
            y = label[i:i+batch_size]
            l1,l2,l3,y1,y2,y3=clf.test(x,mask,y_ent=y[:,:,0],y_pos=y[:,:,1],y_dep=y[:,:,2])
            i += batch_size
            dl.append(l1+l2+l3)
            pent.append(y1.flatten())
            ppos.append(y2.flatten())
            pdep.append(y3.flatten())
            tent.append(y[:,:,0].flatten())
            tpos.append(y[:,:,1].flatten())
            tdep.append(y[:,:,2].flatten())
        tent, pent = unpad(tent,pent)
        tpos, ppos = unpad(tpos,ppos)
        tdep, pdep = unpad(tdep,pdep)
        
        acc = accuracy_score(y_true=tent,y_pred=pent)+accuracy_score(y_true=tpos,y_pred=ppos)+accuracy_score(y_true=tdep,y_pred=pdep)
        
        tent = (tent==ent['OBS']) * tent / ent['OBS']
        pent = (pent==ent['OBS']) * pent / ent['OBS']

        tall.extend(tent)
        pall.extend(pent)
        lp = precision_score(y_true=tent, y_pred=pent)
        lr = recall_score(y_true=tent, y_pred=pent)
        lf1 = f1_score(y_true=tent, y_pred=pent)
        
        writer.add_scalar('test loss', np.mean(dl),e*len(test_corpus)+fi)
        writer.add_scalar('test acc', acc,e*len(test_corpus)+fi)
        writer.add_scalar('loc precision', np.mean(lp), e*len(test_corpus)+fi)
        writer.add_scalar('loc recall',np.mean(lr),e*len(test_corpus)+fi)
        writer.add_scalar('loc f1-score',np.mean(lf1),e*len(test_corpus)+fi)
        
        print(f'test epoch: {e}, doc: {fi}, loss: {np.mean(dl)}, acc: {acc}, precision: {lp:.2f}, recall: {lr:.2f}, f1-score: {lf1:.2f}')
    print('-'*18)

In [ ]:
result = []
print(len(test_corpus))
for fi,f in enumerate(test_corpus):
    corpus = f[0]
    label = lmap(lambda x: np.array(x), f[1])
    label = pad(label,dim=3)
    pent = []
    ppos = []
    pdep = []
    tent = []
    tpos = []
    tdep = []
    i=0
    while i<label.shape[0]:
        if label.shape[0]==0:break
        x = corpus['elmo_representations'][0][i:i+batch_size]
        mask =  corpus['mask'].type(torch.uint8)[i:i+batch_size]
        y = label[i:i+batch_size]
        y1,y2,y3=clf.inference(x,mask)
        i+=batch_size
        pent.append(y1.flatten())
        ppos.append(y2.flatten())
        pdep.append(y3.flatten())
        tent.append(y[:,:,0].flatten())
        tpos.append(y[:,:,1].flatten())
        tdep.append(y[:,:,2].flatten())
    mask = corpus['mask'].numpy().flatten()
    pent = mask_unpad(mask,pent)
    ppos = mask_unpad(mask,ppos)
    pdep = mask_unpad(mask,pdep)
    pent = (pent==ent['OBS'])*pent/ent['OBS']
    result.append(pent)
np.save(WORK_DIR+'result', result)

# Postprocess

In [ ]:
import sys
import argparse
import spacy
import pickle
import codecs
from tqdm import tqdm
import numpy as np
import torch
from os import listdir, makedirs
from os.path import join, exists

nlp = spacy.load('en_core_web_lg')

In [ ]:
test_files = txt_files[train_size:]

labels = np.load(WORK_DIR + 'result.npy')

for findex, txt_file in enumerate(test_files):
    with open(WORK_DIR + txt_file, 'r') as f:
        doc = f.read()
        doc = nlp(doc)
    #print(len(doc))
    label = labels[findex]
    assert len(label) == len(doc)

    for tid, token in enumerate(doc):
        if label[tid] == 1:
            print(token, 'OBS')
        else:
            print(token)

In [ ]:
p = precision_score(y_true=tall, y_pred=pall)
r = recall_score(y_true=tall, y_pred=pall)
f1 = f1_score(y_true=tall, y_pred=pall)
print(f'precision: {p:.2f}, recall: {r:.2f}, f1-score: {f1:.2f}')